## **LinkedIn User Ranker Beta 1.0**
Nir Miller

This is the beta for the LinkedIn User Rank. To run this program insert your org_id into the org_id variable and hit Runtime -> Run all

This program will go through your LinkedIn posts and analyze your most prominent commentors, ranking them by several criteria including : comment history, job title, and relative interest, providing a accurate dipication of customer interest.

# Set up

## Pip Installs

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!pip install transformers
!pip install --user -U nltk
!pip install -U pip setuptools wheel
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Imports

In [ ]:
import spacy
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import nltk
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
import math
import requests
from requests import Session
from datetime import datetime

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()


## Downloads / Load Models

In [ ]:

# Semantic Analyzer
!python -m spacy download en_core_web_lg

nlp = spacy.load("en_core_web_lg")

# Sentiment Analyzer
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# Access Token (Hide)
access_token = 'xxxxxx'

# Temporary Datacenter (Hidden)



/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-09 05:40:52.164089: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-09 05:40:52.164237: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-09 05:40:52.164262: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

# Methods

## Comments Analyzer Methods

In [ ]:
# Returns a sentiment score from 0-1

def sentiment_score(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict

In [ ]:
# Returns a semantic similiarty score from 0-1
# [pos, nuetral, negative]

def semantic_score(example):

  test_sentence = nlp(example)

  s1 = nlp('I love your product, please come speak at our conference')
  s2 = nlp('I sorta want your product, you are good at speaking')
  s3 = nlp('I hate your product, you suck at speaking')

  l = [s1.similarity(test_sentence), s2.similarity(test_sentence), s3.similarity(test_sentence)]
  scaled_l = [0, 0 ,0]
  for i in range(len(l)):
    scaled_l[i] = ((l[i] - min(l)) / (max(l) - min(l)))*((max(l) - min(l))+min(l))

  return scaled_l



In [ ]:
# Returns as a score from 0-1, if its the longest comment on a post
def length_score(example):
  return len(example.split())

def maxlength_score(example, max_length):

  return len(example.split())/max_length

In [ ]:
# Returns # of '!' and '?' all in one value

def punctuation_score(example):
  count = 0
  for i in range(len(example)):
    if example[i] == '!' or example[i] == '?' :
      count+=1

  return count

In [ ]:
def final_comment_score(example):

  number_char = float(len(example))
  number_words = float(length_score(example))
  punc_score = float(punctuation_score(example))


  sent_score = sentiment_score(example)
  sem_score = semantic_score(example)

  #print(number_char, number_words, punc_score, sent_score, sem_score)
  return (punc_score*number_words)/(number_char - number_words) * ((sent_score['roberta_pos'] - sent_score['roberta_neg'])/(abs(sent_score['roberta_pos'] - sent_score['roberta_neg']))) + sem_score[0] - sem_score[2]

## Job Analyzer

In [ ]:
def job_title_score(example):
   with open('/content/drive/MyDrive/LinkedInScraper/Dataset/job_title_levels.txt') as f:
    lines = list(f.readlines())
    highest_title = lines[-1]
    specific_title = "Employee"
    nouns = []
    cleaned_example = nlp(removeSpecialCharacter(example))
    for token in cleaned_example:
        if token.pos_ == 'NOUN' or token.pos_ == 'PROPN':
          nouns.append(token.text)
    #print(nouns)
    for n in nouns:
      max_score = 0
      title = ''
      for l in lines:
        if n.lower() in l.lower():
          title = l
          specific_title = n
      try:
        if lines.index(title) < abs(lines.index(highest_title)):
          highest_title = title
          specific_title = n
      except:
        print("", end="")
    #print(highest_title)
    return (len(lines) - lines.index(highest_title))/len(lines), specific_title

In [ ]:
def removeSpecialCharacter(s):
  characters= '@_!#$%^&*()<>?/\|}{~:]-.'
  a = s
  for i in characters:
    a = a.replace(i, " ")
  return a

## LinkedIn API Methods

In [ ]:
def get_person_data(user_id):
  url = f'https://api.linkedin.com/v2/people/(id:{user_id})'
  headers = {'Authorization' : f'Bearer {access_token}',  'LinkedIn-Version' : '202302', 'X-RestLi-Protocol-Version':'2.0.0'}
  user = requests.get(url, headers=headers)
  return user.json()


In [ ]:
def get_commented_user(share):
  url = f'https://api.linkedin.com/rest/socialActions/{share}/comments?projection=(elements(*(*,actor~(*,profilePicture(displayImage~:playableStreams)))))'
  header = {'Authorization' : f'Bearer {access_token}',  'LinkedIn-Version' : '202301'}

  r = requests.get(url, headers=header)
  return r.json()

In [ ]:

def get_user(user_id):
  url = f'https://api.linkedin.com/v2/people/(id:{user_id})?projection=(id,localizedFirstName,localizedLastName)'
  headers = {'Authorization' : f'Bearer {access_token}',  'LinkedIn-Version' : '202302', 'X-RestLi-Protocol-Version':'2.0.0'}

  user = requests.get(url, headers=headers)
  return user.json()


In [ ]:
def get_posts(org_urn):
  posts_url = f'https://api.linkedin.com/v2/shares?q=owners&owners={org_urn}&sortBy=LAST_MODIFIED&sharesPerOwner=10'
  url = f'https://api.linkedin.com/rest/posts?author={encode_tag(org_urn)}&q=author&count=10'
  headers = {'Authorization' : f'Bearer {access_token}',  'LinkedIn-Version' : '202302', 'X-RestLi-Protocol-Version':'2.0.0', 'X-RestLi-Method' :'FINDER'}
  posts = requests.get(url, headers=headers)
  return posts.json()



In [ ]:
def get_comments(urn):

  comment_url = f'https://api.linkedin.com/rest/socialActions/{urn}/comments'
  headers = {'Authorization' : f'Bearer {access_token}',  'LinkedIn-Version' : '202302'}

  comments = requests.get(comment_url, headers=headers)
  return comments.json()



In [ ]:
def get_replies(urnComments):
  replies_url = f'https://api.linkedin.com/rest/socialActions/{urnComments}/comments'
  replies = requests.get(replies_url, headers=headers)
  headers = {'Authorization' : f'Bearer {access_token}',  'LinkedIn-Version' : '202302'}

  return replies.json()


In [ ]:
def encode_tag(tag):

  return tag.replace(':', '%3A')

In [ ]:
#name_id is id of person or org, data_type is the type of data you want inserted into the datacenter, and data is the data you want to insert
#data can be either a list of datapoints or singular datapoints
def change_data(datacenter, name_id, data_type, new_data):

  if not name_id in list(datacenter.keys()):
    data_points = {'id' : '', 'id_type' : '','name' : '', 'vanity_name' : '', 'job' : '', 'location' : '', 'comment_history': [], 'number_of_comments' : 0, 'number_of_reactions' : 0, 'head_line' : '', 'number_of_likes' : 0}
    datacenter.update({name_id : data_points})

  if len(data_type) > 1:
    for i in range(len(data_type)):
          datacenter[name_id][data_type[i]] = new_data[i]

  else:
      datacenter[name_id][data_type[0]] = new_data





In [ ]:
from logging import exception
def getCommentData(datacenter, comments):

      print(comments)
      for com in comments['elements']:
          print(com)
          id_type = ''
          # person or orginization
          if com['actor'][7:13] == 'person':
              name_id = com['actor'][-10:]
              id_type = 'person'
          else:
              name_id = com['actor'][-8:]
              id_type = 'orginization'
              continue

          if not name_id in list(datacenter.keys()):
            try:
              data_points = {'id' : '', 'id_type' : '','name' : '', 'vanity_name' : '', 'job' : '', 'location' : '', 'comment_history': [], 'number_of_comments' : 0, 'number_of_reactions' : 0, 'head_line' : '', 'number_of_likes' : 0}
              datacenter.update({name_id : data_points})

              datacenter[name_id]['id'] = name_id
              datacenter[name_id]['id_type'] = id_type

              user = get_person_data(name_id)
              print(user)
              datacenter[name_id]['name'] = user['firstName']['localized']['en_US'] + " " + user['localizedLastName']
              datacenter[name_id]['head_line'] = user['localizedHeadline']
              datacenter[name_id]['vanity_name'] = user['vanityName']
              job_score, job_title = job_title_score(datacenter[name_id]['head_line'])
              datacenter[name_id]['job'] = job_title
            except:
              print(exception)

          #Assuming person commented and has already been insterted into datacenter:

          datacenter[name_id]['number_of_comments'] += 1
          datacenter[name_id]['number_of_reactions'] += 1

          datacenter[name_id]['comment_history'].append(com['message']['text'])


In [ ]:
def getPostData(datacenter, org_urn):
  posts = get_posts(org_urn)
  i = 0
  for e in posts['elements']:

    print("Post ", i)
    i+=1
    comment_urn = e['id']
    #time_stamp = e['lastModified']
    if 'ugcPost' in comment_urn or 'share' in comment_urn:
        comments = get_comments(comment_urn)
        getCommentData(datacenter, comments)



In [ ]:
def average_score_comments(comment_history):
  history_length = len(comment_history)
  score_sum = 0
  for comment in comment_history:
    score_sum += final_comment_score(comment)

  return score_sum/history_length




In [ ]:
def final_score(user_number_of_comments, job_title_score, average_comment_interest_score):
  #print(user_number_of_comments, job_title_score, average_comment_interest_score)
  return ((1+(user_number_of_comments/10) + job_title_score**2)**(average_comment_interest_score))


In [ ]:
class User():

  def __init__(self, user_tag = '', user_data=None, user_criteria_scores=None):
    self.user_tag = user_tag
    self.user_data=user_data
    self.vanity_name = user_data['vanity_name']
    self.name = user_data['name']
    self.job = user_data['job']
    self.user_criteria_scores=user_criteria_scores


In [ ]:
def return_sorted_users(datacenter):
  users = list(datacenter.keys())

  user_list = []

  for user in users:
    user_data = datacenter[user]
    user_comment_history = user_data['comment_history']
    user_tag = user_data['head_line']
    user_number_of_comments = user_data['number_of_comments']


    job_score, job_title = job_title_score(user_tag)
    average_comment_interest_score = average_score_comments(comment_history=user_comment_history)
    score = final_score(user_number_of_comments, job_score, average_comment_interest_score)

    new_user = User(user_tag, user_data, user_criteria_scores=score)
    user_list.append(new_user)

  return sorted(user_list, key= lambda x : 1/x.user_criteria_scores)

In [ ]:
def upload_data_gs(creds, link, sorted_users_data):


  gc = gspread.authorize(creds)
  #sh = gc.create('/content/drive/MyDrive/LinkedInScraper/Dataset/UserData_Always_Empty_When_Not_Running/test_data')

  # Open our new sheet and add some data.
  worksheet = gc.open(link).sheet1
  sorted_users= sorted_users_data


  #worksheet.clear()

  #Name, Vanity Name for LinkedIn, Occupation/Title, Interest Score, LinkedIn profile link
  for index in range(len(sorted_users)):
    data = sorted_users[index]
    index_id = f'A{index+1}:E{index+1}'
    cell_list = worksheet.range(index_id)

    input_data = [str(data.name), str(data.vanity_name), str(data.job), str(data.user_criteria_scores), f"https://www.linkedin.com/in/{str(data.vanity_name)}/"]
    for i in range(len(cell_list)):
      cell_list[i].value = input_data[i]
    worksheet.update_cells(cell_list)

  #worksheet.clear()

# Run

In [ ]:
datacenter = dict()

org_urn = 'urn:li:organization:90442007'

link = '/content/drive/MyDrive/LinkedInScraper/Dataset/UserData_Always_Empty_When_Not_Running/test_data'


# Gets Post Data and uploads it to datacenter
getPostData(datacenter, org_urn)

# Sorts data in datacenter based on several inputted criteria
sorted_users = return_sorted_users(datacenter)

# Temporarly Uploads to google sheets for viewing.
upload_data_gs(creds, link, sorted_users)


Post  0
{'paging': {'start': 0, 'count': 10, 'links': [{'type': 'application/json', 'rel': 'next', 'href': '/rest/socialActions/urn:li:share:7029714598974955520/comments?count=10&start=10'}], 'total': 13}, 'elements': [{'actor': 'urn:li:person:TI2s7kHbUp', 'created': {'actor': 'urn:li:person:TI2s7kHbUp', 'time': 1677912570746}, 'lastModified': {'actor': 'urn:li:person:TI2s7kHbUp', 'time': 1677912570746}, 'id': '7037675407130714112', '$URN': 'urn:li:comment:(urn:li:activity:7029714599629262848,7037675407130714112)', 'message': {'attributes': [], 'text': 'Wow I wont this product right NOW!!!!!!!!!'}, 'object': 'urn:li:activity:7029714599629262848'}, {'actor': 'urn:li:person:xqJMgbc0zf', 'created': {'actor': 'urn:li:person:xqJMgbc0zf', 'time': 1677830204829}, 'lastModified': {'actor': 'urn:li:person:xqJMgbc0zf', 'time': 1677830204829}, 'id': '7037329939435573248', '$URN': 'urn:li:comment:(urn:li:activity:7029714599629262848,7037329939435573248)', 'message': {'attributes': [], 'text': 'sup

In [ ]:
datacenter

In [ ]:
for us in sorted_users:
  print(us.user_data['id'] + ' ' + us.user_data['name'] + ' ' + str(us.user_criteria_scores))

xqJMgbc0zf Natalee Chu 1.015501579214744
TI2s7kHbUp Nir Miller 0.6387615326990813


In [ ]:
datacenter = dict()

urn = 'urn:li:ugcPost:6876175560398168064'

link = '/content/drive/MyDrive/LinkedInScraper/Dataset/UserData_Always_Empty_When_Not_Running/test_data'

# Gets Post Data and uploads it to datacenter
comments = get_comments(urn)
getCommentData(datacenter, comments)

# Sorts data in datacenter based on several inputted criteria
sorted_users = return_sorted_users(datacenter)

# Temporarly Uploads to google sheets for viewing.
upload_data_gs(creds, link, sorted_users)


{'message': 'member is unauthorized on urn:li:ugcPost:6876175560398168064', 'status': 403}


KeyError: ignored